<a href="https://colab.research.google.com/github/aadishchopra/NLP/blob/main/Custom_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following the tutorial and making some changes for better understanding

https://www.tensorflow.org/tutorials/customization/custom_layers

# Custom layers

We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.


In [1]:
import tensorflow as tf

In [3]:
print(tf.config.list_physical_devices('GPU'))

[]


## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.


In [23]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(5, input_shape=(None, 5))

In [10]:
# Shape is same as above

tf.keras.Input(shape=[5])



<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'input_5')>

The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [43]:
# To use a layer, simply call it.

# We can see that the output of the layer is driven by the first parameter which is 3 in the case below
layer = tf.keras.layers.Dense(3, input_shape=(None, 6))
layer(tf.zeros([2,2]))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [44]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_17/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.7176936 ,  0.22736311,  0.00438464],
        [ 1.0569317 , -0.91597074,  0.4162413 ]], dtype=float32)>,
 <tf.Variable 'dense_17/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [45]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_17/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.7176936 ,  0.22736311,  0.00438464],
        [ 1.0569317 , -0.91597074,  0.4162413 ]], dtype=float32)>,
 <tf.Variable 'dense_17/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>)

## Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

1. `__init__` , where you can do all input-independent initialization
2. `build`, where you know the shapes of the input tensors and can do the rest of the initialization
3. `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [58]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super().__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    print(input_shape[-1])
    self.kernel = self.add_weight("kernel",shape=[int(input_shape[-1]),self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [60]:
layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [64]:
layer.kernel

<tf.Variable 'my_dense_layer_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 1.9883674e-01, -6.2381953e-01,  5.9318835e-01,  2.2790229e-01,
        -3.8789001e-01, -1.7817155e-01,  1.7820609e-01, -6.2940854e-01,
        -3.2543856e-01, -4.5955527e-01],
       [-4.1260898e-02,  4.7378105e-01,  7.2501957e-02,  3.2033587e-01,
         5.0678593e-01, -3.7829635e-01,  5.3176409e-01,  4.9491769e-01,
         1.7452067e-01, -1.2058878e-01],
       [ 8.6546957e-02, -3.6178827e-02, -6.7790747e-02,  2.9609984e-01,
        -1.3091213e-01, -2.5411063e-01, -4.0848315e-02,  5.6584162e-01,
        -6.2058121e-01,  2.1401459e-01],
       [ 3.3978194e-01,  2.9321057e-01,  5.0285548e-01, -2.1860182e-02,
         1.1649275e-01,  8.1598759e-05,  5.8825368e-01,  4.3143052e-01,
        -2.0475447e-01,  2.0307904e-01],
       [-1.9560203e-01,  1.2986034e-01, -6.0092145e-01, -1.9268516e-01,
         2.6648712e-01, -6.1309588e-01, -4.6184301e-02,  2.2133631e-01,
        -5.9181982e-01,  8.5476339e-02]

In [61]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer_3/kernel:0']


Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in `tf.keras.layers`, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!

## Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see [Custom Keras layers and models](https://www.tensorflow.org/guide/keras/custom_layers_and_models) for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [ ]:
block.layers

In [ ]:
len(block.variables)

In [ ]:
block.summary()

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()

# Next steps

Now you can go back to the previous notebook and adapt the linear regression example to use layers and models to be better structured.